### Uniswap swap formula
$x\cdot y \leq (x + (1 - f)\cdot\Delta x)\cdot(y - \Delta y)$

$\Delta x = \large\frac{x\cdot \Delta y}{(y-\Delta y)\cdot(1-f)} = \frac{m\cdot x\cdot \Delta y}{(y-\Delta y)\cdot(m-m\cdot f)}$

### Uniswap dump formula
$\Delta y = \large\frac{-y\cdot(2-f) + \sqrt{y^2\cdot(2-f)^2 - 4\cdot(1-f)\cdot(y^2-\frac{k}{r})}}{2\cdot(1 - f)}$
  
$\Delta y = y \cdot (\sqrt\frac{1}{r}-1)$
### Comp target ratio formula
#### Positive rebalance
$r = \large\frac{p_{USDT}\cdot(x_{USDT} + \Delta x_{USDT})}{p_{ETH}\cdot(y_{ETH} + \Delta y_{ETH})}$

In [123]:
raw_data = '''
0x0000000000000000000000000000000000000000000000000000000000000100000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000a83fb0a237c6a9c3
12584380000000000000000000000000000000000000000000000001bc17021cc84f67d00000000000000000000000000000
00000000000000000000de0b6b3a733ffe20000000000000000000000000000000000000000000000000000000060fd1a000
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000f454c00000000000000000000000000000000000000000000000000000000000000e0000000000
00000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000003cf
8df79b35735ebfd50b60000000000000000000000000000000000000000000009a7b3f7277c6476b5ec00000000000000000
00000000000000000000000000000afd007d66771213ace0000000000000000000000000000000000000000000000001bc17
021cc84f67d000000000000000000000000000000000000000000000000000000000000001b44454255473a2064657465637
46564204b20756e69206572726f720000000000
'''
debug_data = bytes.fromhex(''.join(raw_data.splitlines())[2:])

In [124]:
def read_uint(message, word):
    section = message[word*32:(word+1)*32]
    return int.from_bytes(section, 'big')

def read_bytes(message, word):
    shift = read_uint(message, word)
    length = read_uint(message, shift // 32)
    return message[shift+32:shift + 32 + length]

In [125]:
reason = read_bytes(debug_data, 0)
reason_str = read_bytes(reason, 0)
x = read_uint(reason, 1)
y = read_uint(reason, 2)
borrow_amount = read_uint(reason, 3)
amount1 = read_uint(reason, 4)
print(reason_str, x, y, borrow_amount, amount1)

b'DEBUG: detected K uni error' 73710818458006648249995446 45594872483203553605100 3243170398523665496782 2000002999996708477


In [128]:
k = x*y
k

3360835368225379920156776341911766597091882374600

In [134]:
k_new = (y - amount1) * (1000 * x + borrow_amount * 997) // 1000
k_new, k - k_new

(3360835368225379920156776334753150288224699406542, 7158616308867182968058)

In [112]:
x / y

1619.9626548777376

In [115]:
value_keys = list(map(str.strip, 'currentRatio, rebalanceValueUsd, ethBorrow,  valueEth, ethPrice, debtDai, daiPrice'.split(',')))
values = {key: read_uint(message, i) for key, i in zip(value_keys, range(1, 8))}
values

{'currentRatio': 0,
 'rebalanceValueUsd': 3254404881162397851057106007,
 'ethBorrow': 2000002999731070459,
 'valueEth': 999999999864035432,
 'ethPrice': 1627200000,
 'debtDai': 0,
 'daiPrice': 1000780}

In [118]:
(y * 1000) /  (y - values['ethBorrow'])

1000.0439119871717

In [29]:
x_dai = 10 * 1000
y_eth = 10
k = x_dai * y_eth
f = 0.003

p_new = x_dai / y_eth * 0.8
p_new, k, x_dai / y_eth

(800.0, 100000, 1000.0)

In [30]:
import math

In [35]:
a = 1 - f
b = y_eth * (2 - f)
c = y_eth**2 - k / p_new
y_dump = (-b + math.sqrt(b**2 - 4 * a * c)) / (2 * a)
y_dump

1.1821131979056239

In [36]:
x_out = x_dai - k / (y_eth + y_dump * (1 - f))
x_out

1054.3094416754993

In [37]:
x_new = x_dai - x_out
y_new = y_eth + y_dump

x_new * y_new

100031.72445662013

In [38]:
x_new / y_new

800.0000000000001